In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense,
    Concatenate,
    Input,
    Embedding,
    Lambda,
    TextVectorization,
    LSTM,
    Normalization,
    GlobalAveragePooling2D,
    GlobalAveragePooling1D,
    Conv2D
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np

In [2]:
user_df = pd.read_csv("dataset/users.csv", delimiter=";")
artwork_df = pd.read_csv("dataset/artworks.csv", delimiter=";")

In [3]:
user_df["place"] = user_df["place"].astype(str)
user_df["place"] = user_df["place"].fillna("N/A")

user_df["inscription_date"] = user_df["inscription_date"].astype(str)
user_df["inscription_date"] = user_df["inscription_date"].fillna("N/A")

In [4]:
def preprocess_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image = image.convert("RGB")
        image = image.resize((224, 224))
        image_array = np.array(image)
        image_array = image_array / 255.0
        return image_array
    else:
        raise FileNotFoundError(
            f"Impossibile scaricare l'immagine dall'URL: {image_url}"
        )

In [5]:
def create_triples(user_df, artwork_df):
    triples = []
    for _, user_row in user_df.iterrows():
        user_interactions = artwork_df[artwork_df["author"] == user_row["name"]]
        if len(user_interactions) < 2:
            continue
        Pu = user_interactions.sample(frac=1).iloc[:-1]
        i = user_interactions.sample()
        non_interacted = artwork_df[~artwork_df["author"].isin([user_row["name"]])]
        j = non_interacted.sample()
        triples.append((Pu, i, j))
    return triples

In [6]:
def preprocess_data(triples, image_preprocessor):
    processed_triples = []

    for Pu, i, j in triples:
        Pu_images = np.array([image_preprocessor(url) for url in Pu["img"].values])
        i_image = np.array(image_preprocessor(i["img"].values[0]))
        j_image = np.array(image_preprocessor(j["img"].values[0]))
        processed_triples.append((Pu_images, i_image, j_image, Pu, i.iloc[0], j.iloc[0]))

    return processed_triples

In [7]:
embedding_dim = 200
pu_dim = 400
max_text_words = 5000
max_comment_words = 1000
margin = 0.2

In [8]:
name_vectorizer=TextVectorization(max_tokens=max_text_words)
name_vectorizer.adapt(user_df["name"])
place_vectorizer=TextVectorization(max_tokens=max_text_words)
place_vectorizer.adapt(user_df["place"])

inscription_date_vectorizer=TextVectorization(max_tokens=max_text_words)
inscription_date_vectorizer.adapt(user_df["inscription_date"])



followers_list_vectorizer=TextVectorization(max_tokens=max_text_words)
followers_list_vectorizer.adapt(user_df["followers_part_1"])

follow_list_vectorizer=TextVectorization(max_tokens=max_text_words)
follow_list_vectorizer.adapt(user_df["follow_part_1"])

art_title_vectorizer=TextVectorization(max_tokens=max_text_words)
art_title_vectorizer.adapt(artwork_df["title"])

art_author_vectorizer=TextVectorization(max_tokens=max_text_words)
art_author_vectorizer.adapt(artwork_df["author"])

art_date_vectorizer=TextVectorization(max_tokens=max_text_words)
art_date_vectorizer.adapt(artwork_df["date"])

art_comments_vectorizer=TextVectorization(max_tokens=max_comment_words)
art_comments_vectorizer.adapt(artwork_df["comment_1"])





In [9]:
resnet = ResNet50(weights='imagenet', include_top=False, pooling=None)
for layer in resnet.layers:
    layer.trainable = False
def extract_features(image):
    features=resnet(image)
    features = GlobalAveragePooling2D()(features)
    return features

In [10]:
def create_text_embedding(text_input, max_words, output_dim):
    embedding = Embedding(input_dim=max_words, output_dim=output_dim)(text_input)
    pooling_embedding = GlobalAveragePooling1D()(embedding)
    return pooling_embedding

In [11]:
normalization_layer = Normalization()

In [12]:
input_user_name = Input(shape=(1,), name="input_user_name")
input_user_place = Input(shape=(1,), name="input_user_place")
input_user_inscription_date = Input(shape=(1,), name="input_user_inscription_date")
input_user_page_views = Input(shape=(1,), name="input_user_page_views")
input_user_followers = Input(shape=(1,), name="input_user_followers")
input_user_follow = Input(shape=(1,), name="input_user_follow")
input_user_favourites = Input(shape=(1,), name="input_user_favourites")
input_user_comments_made = Input(shape=(1,), name="input_user_comments_made")
input_user_comments_received = Input(shape=(1,), name="input_user_comments_received")
input_user_followers_list = Input(shape=(None,), name="input_user_followers_list")
input_user_follow_list = Input(shape=(None,), name="input_user_follow_list")

normalized_inscription_date = normalization_layer(input_user_inscription_date)
normalized_page_views = normalization_layer(input_user_page_views)
normalized_followers = normalization_layer(input_user_followers)
normalized_follow = normalization_layer(input_user_follow)
normalized_favourites = normalization_layer(input_user_favourites)
normalized_comments_made = normalization_layer(input_user_comments_made)
normalized_comments_received = normalization_layer(input_user_comments_received)

normalized_inscription_date = Dense(embedding_dim, activation='selu')(normalized_inscription_date)
normalized_page_views = Dense(embedding_dim, activation="selu")(normalized_page_views)
normalized_followers = Dense(embedding_dim, activation="selu")(normalized_followers)
normalized_follow = Dense(embedding_dim, activation="selu")(normalized_follow)
normalized_favourites = Dense(embedding_dim, activation="selu")(normalized_favourites)
normalized_comments_made = Dense(embedding_dim, activation="selu")(
    normalized_comments_made
)
normalized_comments_received = Dense(embedding_dim, activation="selu")(
    normalized_comments_received
)

user_features = Concatenate()(
    [
        create_text_embedding(input_user_name, max_text_words, embedding_dim),
        create_text_embedding(input_user_place, max_text_words, embedding_dim),
        create_text_embedding(input_user_inscription_date, max_text_words, embedding_dim),
        normalized_page_views,
        normalized_followers,
        normalized_follow,
        normalized_favourites,
        normalized_comments_made,
        normalized_comments_received,
        create_text_embedding(input_user_followers_list,  max_text_words, embedding_dim),
        create_text_embedding(input_user_follow_list,  max_text_words, embedding_dim),
    ]
)

In [13]:
input_art_title = Input(shape=(8,), name="input_art_title")
input_art_author = Input(shape=(1,), name="input_art_author")
input_art_likes = Input(shape=(1,), name="input_art_likes")
input_art_comments = Input(shape=(None,), name="input_art_comments")
input_art_views = Input(shape=(1,), name="input_art_views")
input_art_date = Input(shape=(1,), name="input_art_date")

art_title_embedding = create_text_embedding(
    input_art_title, max_text_words, embedding_dim
)
art_author_embedding = create_text_embedding(
    input_art_author, max_text_words, embedding_dim
)
art_date_embedding = create_text_embedding(
    input_art_date, max_text_words, embedding_dim
)
art_comments_embedding = create_text_embedding(
    input_art_comments, max_comment_words, embedding_dim
)

art_features = Concatenate()(
    [
        art_title_embedding,
        art_author_embedding,
        normalization_layer(input_art_likes),
        normalization_layer(input_art_views),
        normalization_layer(input_art_date),
        art_comments_embedding,
    ]
)

In [14]:
def custom_reduce_sum(x, y):
    return K.sum(x * y, axis=1)

In [28]:
def custom_triplet_loss(y_true, y_pred, margin=0.2):
    score_i, score_j = tf.split(y_pred, num_or_size_splits=2, axis=-1)

    loss = tf.maximum(0.0, margin + score_j - score_i)

    return tf.reduce_mean(loss)

In [16]:
input_pu = Input(shape=(3,224, 224, 3), name="input_pu")
input_i = Input(shape=(224, 224, 3), name="input_i")
input_j = Input(shape=(224, 224, 3), name="input_j")
pu_features = Lambda(
    lambda x: K.map_fn(
        lambda y: extract_features(y), x
    ),
    output_shape=(None, embedding_dim),
)(input_pu)

i_features = extract_features(input_i)
j_features = extract_features(input_j)

dense_layer_1 = Dense(embedding_dim, activation="selu", name="dense_layer_1")
dense_layer_2 = Dense(embedding_dim, activation="selu", name="dense_layer_2")

reduced_pu = Lambda(
    lambda x: K.map_fn(
        lambda y: dense_layer_2(dense_layer_1(y)), x
    ),
    output_shape=(None, embedding_dim),
)(pu_features)


reduced_i = dense_layer_2(dense_layer_1(i_features))
reduced_j = dense_layer_2(dense_layer_1(j_features))

concat_i= Concatenate()([reduced_i, user_features, art_features])
concat_j= Concatenate()([reduced_j, user_features, art_features])


dense_comb_i= Dense(embedding_dim, activation="selu", name="dense_comb_i")(concat_i)
dense_comb_j= Dense(embedding_dim, activation="selu", name="dense_comb_j")(concat_j)


average_pooled_pu = Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)
max_pooled_pu = Lambda(lambda x: K.max(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)

pooled_pu = Concatenate()([average_pooled_pu, max_pooled_pu])


pu_dense_1 = Dense(300, activation="selu", name="pu_dense_1")(pooled_pu)
pu_dense_2 = Dense(200, activation="selu", name="pu_dense_2")(pu_dense_1)

final_pu = Dense(200, activation="selu", name="pu_dense_3")(pu_dense_2)

In [38]:
score_i = Lambda(lambda x: K.sum(x[0] * x[1], axis=-1))([final_pu, dense_comb_i])
score_j = Lambda(lambda x: K.sum(x[0] * x[1], axis=-1))([final_pu, dense_comb_j])
tf.print(f"Score i calcolato: {score_i.shape}")
tf.print(f"Score j calcolato: {score_j.shape}")


# loss = Lambda(lambda x: custom_triplet_loss(x[0], x[1]))([score_j, score_i])
output_scores = Concatenate()([score_i, score_j])
tf.print("Shape di output_scores:", output_scores.shape)

Score i calcolato: ()
Score j calcolato: ()


ZeroDivisionError: integer modulo by zero

In [30]:
curatornet = Model(
    inputs=[
        input_user_name,
        input_user_place,
        input_user_inscription_date,
        input_user_page_views,
        input_user_followers,
        input_user_follow,
        input_user_favourites,
        input_user_comments_made,
        input_user_comments_received,
        input_user_followers_list,
        input_user_follow_list,
        input_art_title,
        input_art_author,
        input_art_likes,
        input_art_comments,
        input_art_views,
        input_art_date,
        input_pu,
        input_i,
        input_j,
    ],
    outputs=output_scores,
)

curatornet.compile(optimizer="adam", loss=custom_triplet_loss)
curatornet.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_pu            │ (None, 3, 224,    │          0 │ -                 │
│ (InputLayer)        │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, None, 200) │          0 │ input_pu[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_i             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_j             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, None, 200) │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ input_i[0][0],    │
│ (Functional)        │ 2048)             │            │ input_j[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_name     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_place    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_inscrip… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_page_vi… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_followe… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_follow   │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_favouri… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_comment… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_comment… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_followe… │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_user_follow_… │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 32,662,415 (124.60 MB)

 Trainable params: 9,074,700 (34.62 MB)

 Non-trainable params: 23,587,715 (89.98 MB)

In [19]:
def prepare_inputs(processed_triples, users_df, target_image_count=3, expected_length=183):
    inputs = {
        "input_user_name": [],
        "input_user_place": [],
        "input_user_inscription_date": [],
        "input_user_page_views": [],
        "input_user_followers": [],
        "input_user_follow": [],
        "input_user_favourites": [],
        "input_user_comments_made": [],
        "input_user_comments_received": [],
        "input_user_followers_list": [],
        "input_user_follow_list": [],
        "input_art_title": [],
        "input_art_author": [],
        "input_art_likes": [],
        "input_art_comments": [],
        "input_art_views": [],
        "input_art_date": [],
        "input_pu": [],
        "input_i": [],
        "input_j": [],
    }

    for pu_images, i_image, j_image, Pu_meta, i_meta, j_meta in processed_triples:
        user_features = users_df[users_df["name"] == i_meta["author"]].iloc[0]
        # i_image  = np.expand_dims(i_image, axis=0)
        # j_image  = np.expand_dims(j_image, axis=0)
        if pu_images.shape[0] < target_image_count:
            pad_width = target_image_count - pu_images.shape[0]
            pu_images = np.pad(pu_images, ((0, pad_width), (0, 0), (0, 0), (0, 0)), mode='constant')
        elif pu_images.shape[0] > target_image_count:
            pu_images = pu_images[:target_image_count]
        inputs["input_pu"].append(pu_images)
        inputs["input_i"].append(i_image)
        inputs["input_j"].append(j_image)
        inputs["input_user_name"].append(user_features["name"])
        inputs["input_user_place"].append(user_features["place"])
        inputs["input_user_inscription_date"].append(user_features["inscription_date"])
        inputs["input_user_page_views"].append(user_features["number_page_views"])
        inputs["input_user_followers"].append(user_features["number_followers"])
        inputs["input_user_follow"].append(user_features["number_follow"])
        inputs["input_user_favourites"].append(user_features["number_favourites"])
        inputs["input_user_comments_made"].append(user_features["number_comments_made"])
        inputs["input_user_comments_received"].append(
            user_features["number_comments_receveid"]
        )
        inputs["input_user_followers_list"].append(user_features["followers_part_1"])
        inputs["input_user_follow_list"].append(user_features["follow_part_1"])

        inputs["input_art_title"].append(i_meta["title"])
        inputs["input_art_author"].append(i_meta["author"])
        inputs["input_art_likes"].append(i_meta["likes"])
        inputs["input_art_comments"].append(i_meta["comment_1"])
        inputs["input_art_views"].append(i_meta["number_of_views"])
        inputs["input_art_date"].append(i_meta["date"])

    for key in inputs:
        try:
            if not isinstance(inputs[key], np.ndarray):
                inputs[key] = np.array(inputs[key])

            current_length = len(inputs[key])
            if current_length != expected_length:
                print(f"Adjusting {key} from {current_length} to {expected_length}")
                repeat_factor = max(1, expected_length // current_length)
                if repeat_factor > 1:
                    inputs[key] = np.tile(inputs[key], (repeat_factor,) + (1,) * (inputs[key].ndim - 1))
                if len(inputs[key]) > expected_length:
                    inputs[key] = inputs[key][:expected_length]
        except Exception as e:
            print(f"Warning: Could not convert {key} to numpy array: {e}")

    return inputs

In [20]:
def vectorizer_input(inputs):
  inputs["input_user_name"] = name_vectorizer(inputs["input_user_name"])
  inputs["input_user_place"] = place_vectorizer(inputs["input_user_place"])
  inputs["input_user_inscription_date"] = inscription_date_vectorizer(inputs["input_user_inscription_date"])
  inputs["input_user_followers_list"] = followers_list_vectorizer(inputs["input_user_followers_list"])
  inputs["input_user_follow_list"] = follow_list_vectorizer(inputs["input_user_follow_list"])
  inputs["input_art_title"] = art_title_vectorizer(inputs["input_art_title"])
  inputs["input_art_author"] = art_author_vectorizer(inputs["input_art_author"])
  inputs["input_art_comments"] = art_comments_vectorizer(inputs["input_art_comments"])
  inputs["input_art_date"] = art_date_vectorizer(inputs["input_art_date"])
  return inputs


In [21]:
def expand_inputs(inputs):
  inputs["input_user_page_views"] = np.expand_dims(inputs["input_user_page_views"], axis=-1)
  inputs["input_user_followers"] = np.expand_dims(inputs["input_user_followers"], axis=-1)
  inputs["input_user_follow"] = np.expand_dims(inputs["input_user_follow"], axis=-1)
  inputs["input_user_favourites"] = np.expand_dims(inputs["input_user_favourites"], axis=-1)
  inputs["input_user_comments_made"] = np.expand_dims(inputs["input_user_comments_made"], axis=-1)
  inputs["input_user_comments_received"] = np.expand_dims(inputs["input_user_comments_received"], axis=-1)
  inputs["input_art_likes"] = np.expand_dims(inputs["input_art_likes"], axis=-1)
  inputs["input_art_views"] = np.expand_dims(inputs["input_art_views"], axis=-1)
  return inputs

In [22]:
def check_input_types(inputs):
    for key, value in inputs.items():
        print(f"Input key: {key}")
        print(f" - Type: {type(value)}")
        if isinstance(value, np.ndarray):
            print(f" - Dtype: {value.dtype}")
            print(f" - Shape: {value.shape}")
        else:
            print(f" - Not a NumPy array")
        print()

In [23]:
triples = create_triples(user_df, artwork_df)
preprocessed_triples = preprocess_data(triples, preprocess_image)
inputs = prepare_inputs(preprocessed_triples, user_df)
inputs = vectorizer_input(inputs)
inputs = expand_inputs(inputs)


In [31]:
labels= np.zeros((len(triples), 1))
curatornet.fit(x=inputs,y=labels,epochs=10)


Epoch 1/10


ValueError: Dimension size must be evenly divisible by 2 but is 1
	Number of ways to split should evenly divide the split dimension for '{{node compile_loss/custom_triplet_loss/split}} = Split[T=DT_FLOAT, num_split=2](compile_loss/custom_triplet_loss/split/split_dim, compile_loss/custom_triplet_loss/ExpandDims)' with input shapes: [], [?,1] and with computed input tensors: input[0] = <-1>.